# Opérateur d'intérêt : Harris

L'opérateur d'intérêt **Harris** est l'un des détecteurs les plus simples utilisés pour identifier les coins des objets sur une image. 
L'idée est de localiser les **points d'intérêt** où le voisinage présente des **contours** à plusieurs directions. L'idée de base de l'algorithme est de trouver la **différence d'intensité** pour un déplacement de *(u,v)* dans toutes les directions, ce qui est exprimé comme suit :   

![img](https://i.ibb.co/mB6Ck2z/Autocorrelation.png)

L'équation ci-dessus peut encore être approximée en utilisant le développement de Taylor, ce qui nous donne la formule finale :

![img](https://i.ibb.co/TLgJm1X/Apr-s-taylor.png)

Avec

![img](https://i.ibb.co/x8L6s9N/Covariance.png)

Ix et Iy sont les dérivées de l'image dans les directions x et y respectivement. On peut calculer la dérivée en utilisant un filtre de gradient, ou un opérateur des contours (Sobel par exemple).

Finalement, nous trouvons la fonction de Harris R donnée par :

![img](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/8499c5fd1fb4572f17432043824392ba87e027e6.png)

Avec

* ![img](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/cf85281c57f753fa4f883455cc85bed7cdbfaab3.png)  
* ![img](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/1eb05457094cbdec3f8d3e505b87a184e3ed8121.png)  
* Les lambdas sont les valeurs propres de <code>M</code>
* <code>k</code> est le facteur de sensibilité pour séparer les coins des bords, typiquement une valeur proche de zéro

Nous trouvons les coins en utilisant la valeur de <code>R</code> :

![img](https://i.ibb.co/1Qy5qrZ/Reponse-harris.png)

## Algorithme de détection des points d'intérêts : Harris

1. Conversion de l'image couleur en niveaux de gris
2. Calcul des images du gradient <code>Ix</code> et <code>Iy</code>
3. Calcul la matrice de variances covariances du gradient <code>M</code>
4. Calcul de la réponse de Harris <code>R</code>
5. Identification des contours et des coins en analysant <code>R</code>

#### Étape 1 : Convertir l'image en niveaux de gris

In [ ]:
!pip3 install scikit-image

In [ ]:
from skimage.io import imread
from skimage.color import rgb2gray
import os

os.chdir("C:/Users/Anass/Desktop/Corner Detector/Data")
img = imread("Aerial Photo.jpg")
imgGray = rgb2gray(img)
imgGray

#### Étape 2 : Calcul des images du gradient <code>Ix</code> et <code>Iy</code>

Le calcul du gradient peut se faire facilement à l'aide du filtre de Sobel, un opérateur utilisé en traitement d'image pour la détection de contours. Il utilise les matrices de convolution:

![img](https://i.ibb.co/n61Jr6J/Filtre-Sobel.png)

Avec
* <code>A</code> l'image source
* <code>Ix</code> gradient horizontal
* <code>Iy</code> gradient vertical

In [ ]:
from scipy import signal as sig
import numpy as np

def gradient_x(imgGray):
    kernel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    return sig.convolve2d(imgGray, kernel_x, mode='same')

def gradient_y(imgGray):
    kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    return sig.convolve2d(imgGray, kernel_y, mode='same')

I_x = gradient_x(imgGray)
I_y = gradient_y(imgGray)

#### Étape 3 : Calcul la matrice de variances covariances du gradient <code>M</code>

In [ ]:
from scipy import ndimage as ndi

# scipy.ndimage.gaussian_filter(input, sigma)
Ixx = ndi.gaussian_filter(I_x**2, sigma=1)
Ixy = ndi.gaussian_filter(I_y*I_x, sigma=1)
Iyy = ndi.gaussian_filter(I_y**2, sigma=1)

#### Étape 4 : Calcul de la réponse de Harris <code>R</code>

In [ ]:
k = 0.04

# déterminant

detM = Ixx * Iyy - Ixy ** 2

# trace
traceM = Ixx + Iyy
    
harris_response = detM - k * traceM ** 2
harris_response

Les petites valeurs de <code>k</code> entraînent la détection des angles aigus. La réponse <code>R</code> est un tableau de pics de chaque ligne de l'image. Nous pouvons utiliser ces valeurs de crête pour isoler les coins et les bords.

#### Étape 5 : Identification des bords et des coins en utilisant <code>R</code>

* Contour : <code>R</code> < 0
* Coin : <code>R</code> > 0
* PLat : <code>R</code> = 0

In [ ]:
img_copy_for_corners = np.copy(img)
img_copy_for_edges = np.copy(img)

for i, response in enumerate(harris_response):
    for j, r in enumerate(response):
        if r > 0.01:
            # c'est un coin
            img_copy_for_corners[i, j] = [255,0,0]
        elif r < -0.05:
            # c'est un contour
            img_copy_for_edges[i, j] = [0,255,0]

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(img_copy_for_corners, interpolation='nearest', cmap=plt.cm.gray)

**Remarque** : l'algorithme identifie les régions d'intérêts. Pour trouver les coins, nous pouvons utiliser un algorithme de recherche de pic

In [ ]:
import matplotlib.pyplot as plt
from skimage.feature import corner_peaks

corners = corner_peaks(harris_response)
fig, ax = plt.subplots(figsize=(17,17))
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
ax.plot(corners[:, 1], corners[:, 0], '.r', markersize=10)